In [5]:
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [6]:
from envs.rendezvous import Poliastro_env
env=Poliastro_env()

/opt/anaconda3/envs/rl/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float16
  logger.warn(


In [7]:
episodes=10
for episode in range(1,episodes+1):
    
    env.reset()
    done=False
    score=0
    steps=0
    
    while not done:
        
        #env.render()
        action=env.action_space.sample()
        reward, done, ob = env.step(action)
        score+=reward
        steps+=1
        if steps>12000:
            break
        
    print("Episode:{}  Score:{}%  Steps: {}".format(episode, score, steps))


Episode:1  Score:26%  Steps: 4193
Episode:2  Score:77%  Steps: 3
Episode:3  Score:0%  Steps: 12001
Episode:4  Score:26%  Steps: 64
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/anaconda3/envs/rl/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/3n/4ywdnyms5bn5zmtm8ymmb1xw0000gn/T/ipykernel_10748/4101828870.py", line 13, in <module>
    reward, done, ob = env.step(action)
  File "/Users/lucas/Desktop/orbitai/rendezvous/envs/rendezvous.py", line 58, in step
  File "/opt/anaconda3/envs/rl/lib/python3.10/site-packages/poliastro/maneuver.py", line 76, in impulse
    return cls((0 * u.s, dv))
  File "/opt/anaconda3/envs/rl/lib/python3.10/site-packages/poliastro/maneuver.py", line 46, in __init__
    self._dts, self._dvs = self._initialize(
  File "/opt/anaconda3/envs/rl/lib/python3.10/site-packages/astropy/units/decorators.py", line 229, in wrapper
    bound_args = wrapped_signature.bind(*func_args, **func_kwargs)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceba

# ML MODEL

In [ ]:
env=DummyVecEnv([lambda: env])
model=PPO("MlpPolicy", env, verbose=1)